# Week3_2 Assignment

## [BASIC](#Basic) 
- 한국어 코퍼스를 로드해 **WordPiece Tokenzier를 학습**시킬 수 있다.
- 학습된 모델을 로드해 **encoding과 decoding을 수행**할 수 있다. 



### Reference
- [BertWordPieceTokenizer 학습 소개 한국어 블로그](https://monologg.kr/2020/04/27/wordpiece-vocab/)
- [huggingface python train tutorial](https://github.com/huggingface/tokenizers/blob/master/bindings/python/examples/train_bert_wordpiece.py)

In [14]:
import os 
import sys
import pandas as pd
import numpy as np

In [15]:
!pip install tokenizers

In [16]:
import torch
from tokenizers import BertWordPieceTokenizer

In [17]:
# seed
seed = 7777
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [18]:
# device type
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"# available GPUs : {torch.cuda.device_count()}")
    print(f"GPU name : {torch.cuda.get_device_name()}")
else:
    device = torch.device("cpu")
print(device)

# available GPUs : 1
GPU name : Tesla P100-PCIE-16GB
cuda


## Basic

### 데이터 다운로드
- 내 구글 드라이브에 데이터를 다운 받은 후 코랩에 드라이브를 마운트하면 데이터를 영구적으로 사용할 수 있다. 
- [데이터 다운로드 출처](https://ratsgo.github.io/embedding/downloaddata.html)

In [19]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
cd '/content/drive/MyDrive/Classroom'

/content/drive/MyDrive/Classroom


In [21]:
# 한국어 위키피디아 데이터 (토크나이즈되지 않은 텍스트) 로드
#!pip install gdown
#!gdown https://drive.google.com/uc?id=1kUecR7xO7bsHFmUI6AExtY5u2XXlObOG
# !unzip processed.zip

In [22]:
_CUR_DIR = os.path.abspath(os.curdir)
print(f"My current directory : {_CUR_DIR}")
_DATA_DIR = os.path.join(_CUR_DIR, "processed")

My current directory : /content/drive/MyDrive/Classroom


### 한국어 위키피디아 코퍼스로 WordPiece tokenizer 학습
- 한국어 위키 

In [23]:
# processed_wiki_ko.txt 파일 불러오기
import pandas as pd

docs = pd.read_table('/content/drive/MyDrive/Classroom/processed/processed_wiki_ko.txt')

In [24]:
docs

,"제임스 얼 ""지미"" 카터 주니어 (, 1924년 10월 1일 ~ )는 민주당 출신 미국 39번째 대통령 (1977년 ~ 1981년)이다. 지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다. 조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 ""땅콩 농부"" (Peanut Farmer)로 알려졌다. 1962년 조지아 주 상원 의원 선거에서 낙선하나 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주 지사 선거에 낙선하지만 1970년 조지아 주 지사를 역임했다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다. 취임식을 올리는 카터 1976년 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책으로 내세워, 포드를 누르고 당선되었다. 카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다. Carter Begin, Camp David 1978.gif|섬네일|300px|캠프데이비드에서 사다트와 베긴과 함께카터는 이집트와 이스라엘을 조정하여, 캠프 데이비드에서 안와르 사다트 대통령과 메나헴 베긴 수상과 함께 중동 평화를 위한 캠프데이비드 협정을 체결했다. 그러나 이것은 공화당과 미국의 유대인 단체의 반발을 일으켰다. 1979년 백악관에서 양국 간의 평화조약으로 이끌어졌다. 또한 소련과 제2차 전략 무기 제한 협상에 조인했다. 카터는 1970년대 후반 당시 대한민국 등 인권 후진국의 국민들의 인권을 지키기 위해 노력했으며, 취임 이후 계속해서 도덕정치를 내세웠다. 그러나 주 이란 미국 대사관 인질 사건에서 인질 구출 실패를 이유로 1980년 대통령 선거에서 공화당의 로널드 레이건 후보에게 져 결국 재선에 실패했다. 또한 임기 말기에 터진 소련의 아프가니스탄 침공 사건으로 인해 1980년 하계 올림픽에 반공국가들의 보이콧을 내세웠다. 지미 카터는 대한민국과의 관계에서도 중요한 영향을 미쳤던 대통령 중 하나다. 인권 문제와 주한미군 철수 문제로 한때 한미 관계가 불편하기도 했다. 1978년 대한민국에 대한 북한의 위협에 대비해 한미연합사를 창설하면서, 1982년까지 3단계에 걸쳐 주한미군을 철수하기로 했다. 그러나 주한미군사령부와 정보기관·의회의 반대에 부딪혀 주한미군은 완전철수 대신 6,000명을 감축하는 데 그쳤다 . 또한 박정희 정권의 인권 문제 등과의 논란으로 불협화음을 냈으나, 1979년 6월 하순, 대한민국을 방문하여 관계가 다소 회복되었다. 1979년 ~ 1980년 대한민국의 정치적 격변기 당시의 대통령이었던 그는 이에 대해 애매한 태도를 보였고, 이는 후에 대한민국 내에서 고조되는 반미 운동의 한 원인이 됐다. 10월 26일, 박정희 대통령이 김재규 중앙정보부장에 의해 살해된 것에 대해 그는 이 사건으로 큰 충격을 받았으며, 사이러스 밴스 국무장관을 조문사절로 파견했다. 12·12 군사 반란과 5.17 쿠데타에 대해 초기에는 강하게 비난했으나, 미국 정부가 신군부를 설득하는데, 한계가 있었고 결국 묵인하는 듯한 태도를 보이게 됐다. 남수단 독립 국민투표에 업저버 사절단을 지도한 카터 퇴임 이후 민간 자원을 적극 활용한 비영리 기구인 카터 재단을 설립한 뒤 민주주의 실현을 위해 제 3세계의 선거 감시 활동 및 기니 벌레에 의한 드라쿤쿠르스 질병 방재를 위해 힘썼다. 미국의 빈곤층 지원 활동, 사랑의 집짓기 운동, 국제 분쟁 중재 등의 활동도 했다. 카터는 카터 행정부 이후 미국이 북핵 위기, 코소보 전쟁, 이라크 전쟁과 같이 미국이 군사적 행동을 최후로 선택하는 전통적 사고를 버리고 군사적 행동을 선행하는 행위에 대해 깊은 유감을 표시 하며 미국의 군사적 활동에 강한 반대 입장을 보이고 있다. 특히 국제 분쟁 조정을 위해 북한의 김일성, 아이티의 세드라스 장군, 팔레인스타인의 하마스, 보스니아의 세르비아계 정권 같이 미국 정부에 대해 협상을 거부하면서 사태의 위기를 초래한 인물 및 단체를 직접 만나 분쟁의 원인을 근본적으로 해결하기 위해 힘썼다. 이 과정에서 미국 행정부와 갈등을 보이기도 했지만, 전직 대통령의 권한과 재야 유명 인사들의 활약으로 해결해 나갔다. 1978년에 채결된 캠프데이비드 협정의 이행이 지지부진 하자 중동 분쟁 분제를 해결하기 위해 1993년 퇴임 후 직접 이스라엘과 팔레인스타인의 오슬로 협정을 이끌어 내는 데도 성공했다. 행렬 50주년을 맞는 날 빌 클린턴, 버락 오바마 대통령과 함께 (2013년) 1993년 1차 북핵 위기 당시 북한에 대한 미국의 군사적 행동이 임박했으나, 미국 전직 대통령으로는 처음으로 북한을 방문하고 미국과 북 양국의 중재에 큰 기여를 해 위기를 해결했다는 평가를 받았다. 또한 이 때 김영삼 대통령과 김일성 주석의 만남을 주선했다. 하지만 그로부터 수주일 후 김일성이 갑자기 사망하여 김일성과 김영삼의 정상회담은 이루어지지 못했다. 미국의 관타나모 수용소 문제, 세계의 인권문제에서도 관심이 깊어 유엔에 유엔인권고등판무관의 제도를 시행하도록 노력하여 독재자들의 인권 유린에 대해 제약을 하고, 국제형사재판소를 만드는 데 기여하여 독재자들 같은 인권유린범죄자를 재판소로 회부하여 국제적인 처벌을 받게 하는 등 인권 신장에 크나 큰 기여를 했다. 2011년 4월 26일부터 29일까지 북한을 3일간 방문했다. 경제문제를 해결하지 못하고 주 이란 미국 대사관 인질 사건에 발목이 잡혀 실패한 대통령으로 평가를 받지만 이란 사태는 미국 내 이란 재산을 풀어주겠다는 조건을 내세워서 사실상 카터가 해결한 것이었고, 사랑의 집짓기 운동 등으로 퇴임 후에 훨씬 더 존경받는 미국 대통령 중에 특이한 인물로 남았다. 그는 2002년 말 인권과 중재 역할에 대한 공로를 인정받아 노벨 평화상을 받게 되었다. 《진정한 리더는 떠난 후에 아름답다》 저자 지미 카터 《지미 카터》 저자 지미 카터(지식의날개, 2018) 분류:1924년 태어남 분류:1976년 미국 대통령 후보 분류:1980년 미국 대통령 후보 분류:그래미상 수상자 분류:노벨 평화상 수상자 분류:미국 해군의 장교 분류:미국의 침례교도 분류:미국의 노벨상 수상자 분류:미국의 농부 분류:미국의 대통령 분류:미국의 역사 (1964-1980) 분류:미국의 외교관 분류:미국의 인도주의자 분류:미국의 제2차 세계 대전 참전 군인 분류:미국의 진보주의 분류:민주당 (미국)의 정치인 분류:살아있는 사람 분류:스코틀랜드계 미국인 분류:아일랜드계 미국인 분류:잉글랜드계 미국인 분류:조지아 공과대학교 동문 분류:조지아주 출신 정치인 분류:조지아주지사 분류:미국의 회고록 작가 분류:에모리 대학교 교수 분류:미국 해군사관학교 동문 분류:미국 미주리 종합군사학원 동문 분류:타임 올해의 인물 분류:군사 기술자 분류:이란 혁명 관련자"
0,"수학 (數學, Mathematics) 은 양, 구조, 공간, 변화 등의 개념을 다루..."
1,"수학에서 상수 란 그 값이 변하지 않는 불변량으로, 변수의 반대말이다. 물리 상수와..."
2,"The Reader.jpg|섬네일|250px|장오노레 프라고나르 작 ""책 읽는 소녀..."
3,"이 문서는 나라 목록 이며, 전 세계 206개 나라 의 각 현황과 주권 승인 정보를..."
4,in flasks.jpg|섬네일|right|250px|화학은 물질에 대해서 연구하는...
...,...
311231,하인리히 보그트 (1890년 10월 5일 - 1968년 1월 23일)는 독일의 천문...
311232,김경호 (1995년 7월 31일 ~ )는 KBO 리그 두산 베어스의 외야수이다. 2...
311233,"1584 년과 1729 년 사이에 출판 된 ""히브리어 라틴 신성한 성경"" 10 판 ..."
311234,김태권 (金兌權)은 대한민국의 만화가 겸 저술가이다. 《김태권의 십자군 이야기》를 ...


In [25]:
print(f"# wiki documents: {len(docs):,}")

# wiki documents: 311,236


In [27]:
# Word Piece Tokenizer 인스턴스 생성
tokenizer = BertWordPieceTokenizer(
    clean_text=True,
    handle_chinese_chars=True,
    strip_accents=False, # 악센트가 있는 character의 악센트를 제거하려면? (ex. é → e)
    lowercase=False, # 한국어는 대소문자가 없는데 소문자 변환이 필요한지?
)

In [28]:
# train
# files: 'processed_wiki_ko.txt'
# vocab_size: 30,000
# min_frequency: 2
# special_tokens = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"]
# limit_alphabet: 1,000
# wordpieces_prefix: '##'

tokenizer.train(
    files = ['/content/drive/MyDrive/Classroom/processed/processed_wiki_ko.txt'],
    vocab_size = 30000,
    min_frequency = 2,
    show_progress = True,
    special_tokens = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"],
    limit_alphabet = 1000,
    wordpieces_prefix ='##'
)

In [29]:
tokenizer.save_model(".", "wordpiece")

['./wordpiece-vocab.txt']

### Encoding
- 저장된 토크나이즈 파일을 로드해 `BertWordPieceTokenizer` 인스턴스를 생성하고 다음을 수행하자. 
    - 사전(vocab)의 단어 개수를 출력
    - 문장을 토크나이징한 후 토큰 id와 토큰 string을 출력

In [30]:
tokenizer = BertWordPieceTokenizer(
    vocab = "/content/drive/MyDrive/Classroom/wordpiece-vocab.txt",
    lowercase = False,
    strip_accents = False,
)

In [31]:
# 사전 단어 개수 출력
tokenizer.get_vocab_size()

30000

In [46]:
sentence = "안녕하세요. 버트를 사용한 모델입니다."
output = tokenizer.encode(sentence)
print(sentence)
print('=>idx   : %s'%output.ids)
print('=>tokens: %s'%output.tokens)
print('=>offset: %s'%output.offsets)
print('=>decode: %s\n'%tokenizer.decode(output.ids))

안녕하세요. 버트를 사용한 모델입니다.
=>idx   : [2, 7864, 20863, 16, 509, 3371, 5566, 2778, 5757, 16, 3]
=>tokens: ['[CLS]', '안녕', '##하세요', '.', '버', '##트를', '사용한', '모델', '##입니다', '.', '[SEP]']
=>offset: [(0, 0), (0, 2), (2, 5), (5, 6), (7, 8), (8, 10), (11, 14), (15, 17), (17, 20), (20, 21), (0, 0)]
=>decode: 안녕하세요. 버트를 사용한 모델입니다.



In [50]:
text = "안녕하세요. 버트를 사용한 모델입니다."

# 토크나이즈한 후 토큰의 id를 출력하라 
token_ids = tokenizer.encode(text).ids
print(token_ids)

# 토크나이즈한 후 각 토큰(string)을 출력하라.
tokens = tokenizer.encode(text).tokens
print(tokens)

[2, 7864, 20863, 16, 509, 3371, 5566, 2778, 5757, 16, 3]
['[CLS]', '안녕', '##하세요', '.', '버', '##트를', '사용한', '모델', '##입니다', '.', '[SEP]']


In [58]:
# 신조어를 토크나이징할 수 있는지 테스트해보자.
text = "홀리몰리과카몰리"
tokens = tokenizer.encode(text).tokens
print(tokens)

['[CLS]', '홀리', '##몰', '##리', '##과', '##카', '##몰', '##리', '[SEP]']


In [59]:
# 사전에 없는 단어는 어떻게 토크나이즈 되는가?
text = "송나현"
unknown_token_ids = tokenizer.encode(text).ids # 토큰 id
unknown_tokens = tokenizer.encode(text).tokens # 토큰
print(unknown_token_ids)
print(unknown_tokens)

[2, 585, 1151, 1083, 3]
['[CLS]', '송', '##나', '##현', '[SEP]']


### Decoding
- 토큰 id를 원래 문장으로 디코딩하자.

In [60]:
# 원래 문장: "안녕하세요. 버트를 사용한 모델입니다."
tokenizer.decode(token_ids)

'안녕하세요. 버트를 사용한 모델입니다.'

In [61]:
# 사전에 없는 단어는 어떻게 디코딩되는가?
tokenizer.decode(unknown_token_ids)

'송나현'